#### Explanation of materialising and loading the historic C360 data

Run following command in terminal to materialise any required timestamp:

`> pb run --end_time <required_time_in_unix_timestamp>`

Steps to fetch that past materialised data:
1. Connect to warehouse using ProfilesConnector. The config can be accessed from the siteconfig file without changing any format.
2. Load the material registry table, and filter only the successful runs.
3. From material registry table, apply following filters to get the model_hash and seq_no of the right materialisation
    - `model_type` = 'entity_traits_360'
    - `end_ts` = '<unix time from above, after converting to timestamp - see below>'
4. There may be multiple materialisations of same end ts. In such cases, it returns the most recently created materialisation details - seq_no and the model hash are the attributes that define a materialisation
5. For each materialisation, there are multiple views providing the same C360 view with different identifiers as the primary key (ex: email, user_main_id, anonymous_id etc). By defining the id_type to whatever identifier you choose, we then fetch the appropriate view from warehouse using the model hash and seq_no



In [12]:
# Params for fetching the past data - 
# 1. past_materialisation_timestamp in unix timestamp (the one used in pb run command)
# 2. id type of the C360 view 
# 3. entity name of the profiles project - defined in pb_project.yaml
end_time = 1708905600
id_type = 'anonymous_id'
entity_name = 'user'

In [2]:
from profiles_rudderstack.wh import ProfilesConnector
import yaml
import pandas as pd
import os
import numpy as np
import datetime

In [3]:
homedir = os.path.expanduser("~") 
with open(os.path.join(homedir, ".pb/siteconfig.yaml"), "r") as f:
    creds = yaml.safe_load(f)["connections"]["default"]["outputs"]["dev"]

In [4]:
for k, v in creds.items():
    print(f"{k}: <{k if k != 'type' else 'redshift|bigquery|snowflake etc'}>")

account: <account>
dbname: <dbname>
password: <password>
role: <role>
schema: <schema>
type: <redshift|bigquery|snowflake etc>
user: <user>
warehouse: <warehouse>


In [5]:
connector = ProfilesConnector(config=creds)

In [6]:
material_registry = connector.run_query("select * from material_registry_4")
def safe_parse_json(entry):
    try:
        if isinstance(
            entry, str
        ): 
            entry_dict = eval(entry)
        elif isinstance(
            entry, dict
        ):
            entry_dict = entry

        return entry_dict.get("complete", {}).get("status")
    except:
        return None

material_registry["status"] = material_registry["metadata"].apply(
    safe_parse_json
)
material_registry = material_registry.query("status==2")

In [7]:

dt = datetime.datetime.fromtimestamp(end_time, tz=datetime.timezone.utc).replace(tzinfo=None)
dt = np.datetime64(dt)
material_registry["end_ts_np"] = material_registry["end_ts"].apply(lambda x: x.to_numpy())

In [8]:
shortlisted_df = (material_registry
                  .query("end_ts_np==@dt and model_type=='entity_traits_360'")
                  .filter(["model_name", "model_hash", "seq_no", "begin_ts", "end_ts", "creation_ts"])
                  .sort_values(by='creation_ts', ascending=False)
                  .groupby(['model_name', "seq_no"])['model_hash']
                  .first()
                  .reset_index())

In [13]:
# Get the relevant id type view details

model_details = shortlisted_df.query(f"model_name=='{id_type}_{entity_name}_default_entity_serve_360'").to_dict(orient='records')[0]
model_details

{'model_name': 'anonymous_id_user_default_entity_serve_360',
 'seq_no': 42,
 'model_hash': '2d3f871e'}

In [14]:
material_name = f"material_{model_details['model_name']}_{model_details['model_hash']}_{model_details['seq_no']}"
material_name

'material_anonymous_id_user_default_entity_serve_360_2d3f871e_42'

In [15]:
material_data = connector.run_query(f"select * from {material_name}")
material_data.head()

,anonymous_id,active_days_in_past_365_days,active_days_in_past_7_days
0,dc5b9ccfe186bec939a501bf257b5f5e2c71cb58,20,1.0
1,f2bdc0faa416f4d14122701abf0d22695577e33a,25,NaN
2,d5f4309fcd8fb395f22f5e4d060a7ccad670da35,4,NaN
3,70ae06a3c5cfd24af797c4f608fe929abd0c03e0,76,1.0
4,cbbb220fba85977e4a9b44ae7ff0ec106d2c8351,76,1.0
